<a href="https://colab.research.google.com/github/koklengyeo/aviation/blob/main/steel_producers_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREAMBLE

In [98]:
#FOR FUTURE REFERENCE
#DOWNGRADE PYTHON TO VERSION USED DURING DEVELOPMENT
#!apt-get update -y
#!apt-get install python3.10 python3.10-distutils
#!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
#!update-alternatives --config python3
#!apt-get install python3-pip
#!python3 -m pip install --upgrade pip --user

In [99]:
#FOR FUTURE REFERENCE
#DOWNGRADE PACKAGES TO VERSION USED DURING DEVELOPMENT
#!pip install IPython==7.34.0
#!pip install ipywidgets==7.7.1
#!pip install matplotlib==3.7.1
#!pip install numpy==1.25.2
#!pip install pandas==2.0.3

In [100]:
#IMPORT PACKAGES
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import *
from ipywidgets import *

# USER INPUT FORM (WIDGETS)

In [101]:
#DEFINE WIDGETS USED TO CAPTURE USER INPUTS

#BASE YEAR
#MUST BE BETWEEN 2015 AND PRESENT YEAR
by_widget=widgets.BoundedIntText(
    value=datetime.now().year,
    min=2015,
    max=datetime.now().year,
    step=1,
    description="Base Year",
    disabled=False)

#TARGET YEAR
#MUST BE BETWEEN (PRESENT YEAR+5) AND 2050
ty_widget=widgets.BoundedIntText(
    value=datetime.now().year+5,
    min=datetime.now().year+5,
    max=2050,
    step=1,
    description="Target Year",
    disabled=False)

#CORE BOUNDARY EMISSIONS IN TCO2E
#MUST BE >=0
cbe_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Core Boundary Emissions in tCO2e",
    disabled=False)

#BASE YEAR HOT ROLLED STEEL PRODUCTION IN TONNES
#MUST BE >=0
by_hrs_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Hot Rolled Steel Production in Tonnes",
    disabled=False)

#BASE YEAR SCRAP RATIO
#MUST BE BETWEEN 0 AND 100
by_sr_widget=widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="Scrap Ratio in %",
    disabled=False)

#TYPE OF ACTIVITY FORECAST
af_type_widget=widgets.RadioButtons(
    value=None,
    options=[
        ("Fixed Market Share","fms"),
         ("Target Year Activity","tya")],
    description="Type of Activity Forecast",
    disabled=False)

#TARGET YEAR HOT ROLLED STEEL PRODUCTION IN TONNES
#MUST BE >=0
ty_hrs_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Hot Rolled Steel Production in Tonnes",
    disabled=False)

#TARGET YEAR SCRAP RATIO
#MUST BE BETWEEN 0 AND 100
ty_sr_widget=widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="Scrap Ratio in %",
    disabled=False)

#INVISIBLE BOOLEAN WIDGET TO REGISTER FIRST CLICK OF "CALCULATE TARGET" BUTTON
fc_widget=widgets.Valid(
    value=False,
    layout={"display":"none"})

In [102]:
#ENABLE/DISABLE TY_CFA WIDGET DEPENDING ON AF_TYPE

def update_widgets(*args):
  if af_type_widget.value=="fms":
    #DISABLE
    ty_hrs_widget.value=0
    ty_hrs_widget.disabled=True
  elif af_type_widget.value=="tya":
    #ENABLE
    ty_hrs_widget.disabled=False
af_type_widget.observe(update_widgets,"value")

In [103]:
#DEFINE WIDGETS USED TO DISPLAY TEXT

overall_header=widgets.HTML(
    value="<h1>TARGET SETTING FOR STEEL PRODUCERS</h1>\
    <h3>Version 1.0 (Jul-23)</h3>This is intended for steel producers. Targets \
    are aligned with a 1.5&deg;C scenario.")
basic_info_header=widgets.HTML(
    value="<h2>1. Basic Information</h2>Please indicate the base year and \
    target year. Target must cover a minimum of 5 years and a maximum of 10 \
    years from the year it is submitted to the SBTi for validation.")
by_emissions_header=widgets.HTML(
    value="<h2>2. Base Year Emissions</h2>Please provide the total emissions \
    within the core boundary in the base year.")
by_activity_header=widgets.HTML(
    value="<h2>3. Base Year Activity</h2>Please provide the activity and scrap \
    ratio in the base year.")
ty_activity_header=widgets.HTML(
    value="<h2>4. Target Year Activity</h2>Please indicate whether the target \
    model should assume a constant market share or project future activity \
    from the expected activity in the target year. Please also provide the \
    expected scrap ratio in the target year. Companies are strongly encouraged \
    to increase their scrap ratios.")
ty_activity_footer=widgets.HTML(
    value="Please ensure that all provided information is as complete and \
    accurate as possible. Click the button below to calculate the target.")
results_header=widgets.HTML(
    value="<h2>5. Target Modelling Results</h2>")

#ERROR MESSAGES
no_error=widgets.HTML()
cbe_error=widgets.HTML(
    value="<p style='color:red;'>Core Boundary Emissions in tCO2e must be \
      provided.</p>")
hrs_error=widgets.HTML(
    value="<p style='color:red;'>Hot Rolled Steel Production in Tonnes must be \
      provided.</p>")
af_type_error=widgets.HTML(
    value="<p style='color:red;'>Type of Activity Forecast: Please select the \
      applicable option.</p>")
user_inputs_error=widgets.HTML(
    value="<p style='color:red;'>Information provided incomplete and/or \
      invalid. Please check and try again.</p>")
no_target_error=widgets.HTML(
    value="<p style='color:red;'>No target calculated. The base year intensity \
      is already below the 2050 level in the 1.5&deg;C scenario.</p>")

In [104]:
#WIDGET LAYOUT & STYLE

#SELECTION WIDGETS
layout={"width":"1100px"}
style={"description_width":"600px"}
af_type_widget.layout=layout
af_type_widget.style=style

#YEAR WIDGETS
layout={"width":"700px"}
style={"description_width":"600px"}
by_widget.layout=layout
by_widget.style=style
ty_widget.layout=layout
ty_widget.style=style

#EMISSIONS WIDGETS
layout={"width":"800px"}
style={"description_width":"600px"}
cbe_widget.layout=layout
cbe_widget.style=style

#ACTIVITY WIDGETS
layout={"width":"800px"}
style={"description_width":"600px"}
by_hrs_widget.layout=layout
by_hrs_widget.style=style
ty_hrs_widget.layout=layout
ty_hrs_widget.style=style

#PERCENTAGE WIDGETS
layout={"width":"700px"}
style={"description_width":"600px"}
by_sr_widget.layout=layout
by_sr_widget.style=style
ty_sr_widget.layout=layout
ty_sr_widget.style=style

#TEXT WIDGETS
layout={"width":"1100px"}
overall_header.layout=layout
basic_info_header.layout=layout
by_emissions_header.layout=layout
by_activity_header.layout=layout
ty_activity_header.layout=layout
ty_activity_footer.layout=layout
results_header.layout=layout
cbe_error.layout=layout
hrs_error.layout=layout
af_type_error.layout=layout
user_inputs_error.layout=layout
no_target_error.layout=layout

# USER INPUT FORM (INTERACTIVITY)

In [105]:
#USER INPUT FORM - BASIC INFORMATION

def basic_info_check(by,
                     ty,
                     fc):
  return True
basic_info_input=interactive(
    basic_info_check,
    by=by_widget,
    ty=ty_widget,
    fc=fc_widget)

In [106]:
#USER INPUT FORM - BASE YEAR EMISSIONS

def by_emissions_check(cbe,
                       fc):
  by_emissions_valid=True
  update_display(no_error,display_id="by_emissions_check")
  #CBE - MUST BE PROVIDED
  if cbe==0:
    by_emissions_valid=False
    if fc:
      display(cbe_error,display_id="by_emissions_check")
  return by_emissions_valid
by_emissions_input=interactive(
    by_emissions_check,
    cbe=cbe_widget,
    fc=fc_widget)

In [107]:
#USER INPUT FORM - BASE YEAR ACTIVITY

def by_activity_check(by_hrs,
                      by_sr,
                      fc):
  by_activity_valid=True
  update_display(no_error,display_id="by_activity_check")
  #BY_HRS
  if by_hrs==0:
    by_activity_valid=False
    if fc:
      display(hrs_error,display_id="by_activity_check")
  return by_activity_valid
by_activity_input=interactive(
    by_activity_check,
    by_hrs=by_hrs_widget,
    by_sr=by_sr_widget,
    fc=fc_widget)

In [108]:
#USER INPUT FORM - TARGET YEAR ACTIVITY

def ty_activity_check(af_type,
                      ty_hrs,
                      ty_sr,
                      fc):
  ty_activity_valid=True
  update_display(no_error,display_id="ty_activity_check")
  #AF_TYPE - USER MUST SELECT APPLICABLE OPTION
  if af_type is None:
    ty_activity_valid=False
    if fc:
      display(af_type_error,display_id="ty_activity_check")
  if af_type=="tya":
    #TY_HRS - MUST BE PROVIDED
    if ty_hrs==0:
      ty_activity_valid=False
      if fc:
        display(hrs_error,display_id="ty_activity_check")
  return ty_activity_valid
ty_activity_input=interactive(
    ty_activity_check,
    af_type=af_type_widget,
    ty_hrs=ty_hrs_widget,
    ty_sr=ty_sr_widget,
    fc=fc_widget)

# TARGET MODEL (DATA & PARAMETERS)

In [109]:
#IMPORT DATA

#IMPORT CORE BOUNDARY IMAGE
_="<img src='https://koklengyeo.github.io/migration/steel_core_boundary.png' \
width=1100>"
core_boundary=widgets.HTML(
    value=_)

path="https://raw.githubusercontent.com/koklengyeo/migration/main/"

#IMPORT SECTOR DATA
steel=pd.read_csv(
    path+"steel.csv",
    decimal=",",
    index_col=0,
    sep=";")
steel.columns=steel.columns.map(int)

In [110]:
#DEFINE PARAMETERS

pm={#INTENSITY UNIT CONVERSION
    #INTENSITY=IUC*EMISSIONS/ACTIVITY (IN SECTOR DATA UNITS)
    "iuc":1,
    #YEAR AXIS
    "year":pd.Series(range(2015,2051)).set_axis(range(2015,2051))}

# TARGET MODEL (FUNCTIONS)

In [111]:
#FUNCTION - CHECK IF USER INPUTS SATISFY IMPOSED CONDITIONS

def user_inputs_check():
  if not basic_info_input.result \
  or not by_emissions_input.result \
  or not by_activity_input.result \
  or not ty_activity_input.result:
    user_inputs_valid=False
  else:
    user_inputs_valid=True
  return user_inputs_valid

In [112]:
#FUNCTION - CAPTURE USER INPUTS INTO DICTIONARY

def user_inputs_capture():
  return {"by":basic_info_input.children[0].value,
          "ty":basic_info_input.children[1].value,
          "cbe":by_emissions_input.children[0].value,
          "by_hrs":by_activity_input.children[0].value,
          "by_sr":by_activity_input.children[1].value,
          "af_type":ty_activity_input.children[0].value,
          "ty_hrs":ty_activity_input.children[1].value,
          "ty_sr":ty_activity_input.children[2].value}

In [113]:
#FUNCTION - CONVERT USER INPUTS TO FORM REQUIRED BY MODEL

def user_inputs_convert(ip):
  #CONVERT PERCENTAGES TO PROPORTION
  ip["by_sr"]*=.01
  ip["ty_sr"]*=.01

  return ip

In [114]:
#FUNCTION - SECTOR DECARBONIZATION APPROACH

def sda(
    #SECTOR OR SECTOR SEGMENT NAME (IN SECTOR DATA)
    name=None,
    #BASE YEAR
    by=None,
    #TARGET YEAR
    ty=None,
    #BASE YEAR EMISSIONS (IN SAME UNIT AS SECTOR DATA)
    bye=None,
    #ACTIVITY FORECAST (IN SAME UNIT AS SECTOR DATA)
    af=None,
    #SECTOR DATA
    sector_data=None,
    #INTENSITY UNIT CONVERSION
    #(SECTOR INTENSITY UNIT)*(SECTOR ACTIVITY UNIT)/(SECTOR EMISSIONS UNIT)
    iuc=None):

  #SECTOR EMISSIONS, ACTIVITY & INTENSITY
  output=sector_data.loc[sector_data.index.str.contains(name)]
  for i in ["Emissions","Activity","Intensity"]:
    _=output.index[output.index.str.contains(name+" "+i)].values[0]
    output=output.rename(index={_:"Sector "+i})

  #COMPANY ACTIVITY
  output.loc["Company Activity"]=af

  #SDA (EQUATION 6 IN SDA REPORT)
  #SI TERM
  si=output.loc["Sector Intensity",2050]
  #P TERM
  p=output.loc["Sector Intensity"]-si
  p/=output.loc["Sector Intensity",by]-si
  #D TERM
  if output.loc["Company Activity",by]!=0:
    d=iuc*bye/output.loc["Company Activity",by]-si
  else:
    d=-si
  #M TERM
  m=output.loc["Company Activity",by]/output.loc["Sector Activity",by]
  m/=output.loc["Company Activity"]/output.loc["Sector Activity"]
  m[m>1]=1

  #COMPANY INTENSITY
  output.loc["Company Intensity"]=d*p*m+si
  output.loc["Company Intensity"]=output.loc["Company Intensity"].fillna(0)

  #COMPANY EMISSIONS
  output.loc["Company Emissions"]=\
  output.loc["Company Intensity"]*output.loc["Company Activity"]/iuc
  output.loc["Company Emissions"]=output.loc["Company Emissions"].fillna(0)

  #MASK YEAR<BASE YEAR
  output.loc[output.index.str.contains("Company"),output.columns<by]=np.nan

  #SORT
  output=output.sort_index()

  return output

In [115]:
#FUNCTION - TARGET MODEL

def target_model():

  #ACTIVITY FORECAST
  if ip["af_type"]=="fms":
    af=steel.loc["Total Activity (t)"]*ip["by_hrs"]
    af/=steel.loc["Total Activity (t)",ip["by"]]
  elif ip["af_type"]=="tya":
    af=(pm["year"]-ip["by"])*(ip["ty_hrs"]-ip["by_hrs"])
    af/=(ip["ty"]-ip["by"])
    af+=ip["by_hrs"]
    af.loc[af.index>=ip["ty"]]=af.loc[ip["ty"]]

  #SCRAP RATIO FORECAST
  srf=(pm["year"]-ip["by"])*(ip["ty_sr"]-ip["by_sr"])
  srf/=(ip["ty"]-ip["by"])
  srf+=ip["by_sr"]
  srf.loc[srf.index>=ip["ty"]]=srf.loc[ip["ty"]]

  #SCRAP RATIO-WEIGHTED SECTOR INTENSITY
  sector_data=steel.copy(deep=True)
  sector_data.loc["Total Intensity (tCO2e/t)"]=\
  sector_data.loc["Primary Intensity (tCO2e/t)"]*(1-srf)
  sector_data.loc["Total Intensity (tCO2e/t)"]+=\
  sector_data.loc["Secondary Intensity (tCO2e/t)"]*srf

  #SDA TARGET
  output=sda(name="Total",
             by=ip["by"],
             ty=ip["ty"],
             bye=ip["cbe"],
             af=af,
             sector_data=sector_data,
             iuc=pm["iuc"])

  return output

# RESULTS (FUNCTIONS)

In [116]:
#FUNCTION - SECTOR PLOT

def sector_plot():
  #SECTOR ACTIVITY RELATIVE TO 2015
  ra=steel.loc["Total Activity (t)"]
  ra/=steel.loc["Total Activity (t)",2015]

  #SECTOR EMISSIONS RELATIVE TO 2015
  _=steel.loc["Total Emissions (tCO2e)"]
  _/=steel.loc["Total Emissions (tCO2e)",2015]
  re=_/ra

  #GLOBAL SCRAP SHARE RELATIVE TO 2015
  rs=steel.loc["Total Scrap Ratio (%)"]
  rs/=steel.loc["Total Scrap Ratio (%)",2015]

  #CONVERT TO CHANGE FROM 2015
  ra=100*(ra-1)
  re=100*(re-1)
  rs=100*(rs-1)

  #CHANGE IN SECTOR ACTIVITY OVER TARGET TIMEFRAME
  da=steel.loc["Total Activity (t)",ip["ty"]]
  da/=steel.loc["Total Activity (t)",ip["by"]]
  da=100*(da-1)
  da=str(round(da))+"%"

  #CHANGE IN GLOBAL SCRAP RATIO OVER TARGET TIMEFRAME
  ds=steel.loc["Total Scrap Ratio (%)",ip["ty"]]
  ds/=steel.loc["Total Scrap Ratio (%)",ip["by"]]
  ds=100*(ds-1)
  ds=str(round(ds))+"%"

  #SETUP FIGURE
  fig,ax=plt.subplots()
  fig.set_size_inches(5,4)

  #SECTOR PLOT
  ax.plot(ra,
          color="blue",
          label="Sector Activity")
  ax.plot(re,
          color="orange",
          label="Sector Emissions")
  ax.plot(rs,
          color="purple",
          label="Global Scrap Share")
  #SHADE TARGET TIMEFRAME
  ax.fill_between([ip["by"],ip["ty"]],
                  ax.get_ylim()[0],
                  ax.get_ylim()[1],
                  color="grey",
                  alpha=0.2)
  #ANNOTATE CHANGE OVER TARGET TIMEFRAME
  _="Over the target timeframe (shaded), sector activity is\nprojected to "+\
  "grow by "+da+" and the global scrap share by "+ds+"."
  ax.annotate(_,xy=(0.5,-0.2),xycoords="axes fraction",ha="center",va="top")
  #FORMAT
  ax.grid()
  ax.legend()
  ax.set_title("Sector Projection (1.5$\degree$C Scenario)")
  ax.set_xlabel("Year")
  ax.set_ylabel("Percentage Change Compared to 2015")

  plt.show(block=False)

In [117]:
#FUNCTION - TARGET TABLE

def target_table():
  #FETCH BASE YEAR & TARGET YEAR DATA
  table=target.loc[["Company Emissions",
                    "Company Intensity"],
                     [ip["by"],ip["ty"]]]

  #% REDUCTION
  table["% Reduction ("+str(ip["by"])+"-"+str(ip["ty"])+")"]=\
  -1e2*(table[ip["ty"]]/table[ip["by"]]-1)

  #COMMENTS
  table[" "]=" "
  table.iloc[:,3]=table.iloc[:,3].astype(str)

  if table.iloc[0,0]<table.iloc[0,1]:
    table.iloc[0,3]="Target year emissions shall not be higher than base year \
    emissions."
  if table.iloc[1,0]<table.iloc[1,1]:
    table.iloc[1,3]="Target year intensity shall not be higher than base year \
    intensity."

  #RENAME ROWS
  table.index=["Company Core Boundary Emissions (tCO2e)",
                "Company Core Boundary Intensity (tCO2e/t)"]

  #RENAME COLUMNS
  table=table.rename({ip["by"]:"Base Year ("+str(ip["by"])+")",
                      ip["ty"]:"Target Year ("+str(ip["ty"])+")"},
                     axis=1)

  #ROUNDING
  table.iloc[0,:2]=table.iloc[0,:2].apply(lambda x:round(x))
  table.iloc[1,:2]=table.iloc[1,:2].apply(lambda x:round(x,2))
  table.iloc[:,2]=table.iloc[:,2].apply(lambda x:round(x))

  #PERCENTAGE SIGN
  table.iloc[:,2]=table.iloc[:,2].astype(int).astype(str)+"%"

  #FORMAT DISPLAY
  table=table.style
  #NUMBER OF DECIMAL PLACES
  table=table.format(
      precision=0,
      subset=("Company Core Boundary Emissions (tCO2e)",slice(None)))
  table=table.format(
      precision=2,
      subset=("Company Core Boundary Intensity (tCO2e/t)",slice(None)))
  #CENTRE ALIGN
  table=table.set_properties(**{"text-align":"center"})
  #LEFT ALIGN & RED COMMENTS COLUMN
  table=table.set_properties(subset=[" "],**{"text-align":"left",
                                             "color":"red"})
  #COLUMN HEADING - MAX WIDTH 100PX & CENTRE ALIGN
  #ROW HEADING - RIGHT ALIGN
  table=table.set_table_styles(
      [dict(selector=".col_heading",
            props=[("max-width","100px"),("text-align","center")]),
       dict(selector=".row_heading",
            props=[("text-align","right")])])

  display(table)

In [118]:
#FUNCTION USED IN COMPANY_TARGET_PLOT

#LABEL BASE YEAR & TARGET YEAR
def label_by_ty(series=None,ax=None):
  x=ip["by"]
  y=series[ip["by"]]
  ax.scatter(x,y,color="blue")
  ax.text(x,y," Base Year",color="blue",verticalalignment="bottom")
  x=ip["ty"]
  y=series[ip["ty"]]
  ax.scatter(x,y,color="blue")
  ax.text(x,y," Target Year",color="blue",verticalalignment="bottom")

In [119]:
#FUNCTION - COMPANY TARGET PLOT

def company_target_plot():
  #FETCH DATA (EXCLUDE YEAR<BASE YEAR)
  chart=pd.concat([steel,target])
  chart=chart.iloc[:,chart.columns>=ip["by"]]

  #SETUP FIGURE
  fig,(ax1,ax2)=plt.subplots(1,2,gridspec_kw={"wspace":0.3})
  fig.set_size_inches(10,4)

  #COMPANY EMISSIONS PLOT
  ax1.plot(chart.loc["Company Emissions"],
          color="blue",
          label="Company Emissions")
  #LABEL BASE YEAR & TARGET YEAR
  label_by_ty(series=chart.loc["Company Emissions"],ax=ax1)
  #FORMAT
  ax1.grid()
  ax1.legend()
  ax1.set_title("Company Core Boundary\nEmissions Target")
  ax1.set_xlabel("Year")
  ax1.set_ylabel("Emissions (MTCO2e)")

  #COMPANY & SECTOR INTENSITY PLOT
  ax2.plot(chart.loc["Sector Intensity"],
          color="orange",
          label="Sector Intensity (Company Scrap Ratio)")
  ax2.plot(chart.loc["Primary Intensity (tCO2e/t)"],
           color="orange",
           linestyle="dotted",
           label="Sector Intensity (100% Ore)")
  ax2.plot(chart.loc["Secondary Intensity (tCO2e/t)"],
           color="orange",
           linestyle="dashed",
           label="Sector Intensity (100% Scrap)")
  ax2.plot(chart.loc["Company Intensity"],
          color="blue",
          label="Company Intensity")
  #LABEL BASE YEAR & TARGET YEAR
  label_by_ty(series=chart.loc["Company Intensity"],ax=ax2)
  #FORMAT
  ax2.grid()
  ax2.legend(loc='upper center',bbox_to_anchor=(0.5,-0.15))
  ax2.set_title("Company Core Boundary\nIntensity Target")
  ax2.set_xlabel("Year")
  ax2.set_ylabel("Intensity (gCO2e/RTK)")

  plt.show(block=False)

In [120]:
#FUNCTION - TARGET DATA

def data_table():
  #FETCH DATA (EXCLUDE YEAR<BASE YEAR)
  table=target.loc[["Company Emissions",
                   "Company Intensity"],
                  target.columns>=ip["by"]]

  #RENAME ROWS
  table.index=["Company Core Boundary Emissions (tCO2e)",
              "Company Core Boundary Intensity (tCO2e/t)"]

  #ROUNDING
  table.iloc[:,0]=table.iloc[:,0].apply(lambda x:round(x))
  table.iloc[:,1]=table.iloc[:,1].apply(lambda x:round(x,2))

  #COMMENTS ROW
  table.loc[" "]=" "
  table.loc[" ",ip["by"]]="Base Year"
  table.loc[" ",ip["ty"]]="Target Year"

  #TRANSPOSE
  table=table.transpose()

  #FORMAT DISPLAY
  table=table.style
  #NUMBER OF DECIMAL PLACES
  table=table.format(
      precision=0,
      subset=(slice(None),"Company Core Boundary Emissions (tCO2e)"))
  table=table.format(
      precision=2,
      subset=(slice(None),"Company Core Boundary Intensity (tCO2e/t)"))
  table=table.format(
      na_rep="",
      subset=(slice(None)," "))
  #CENTRE ALIGN
  table=table.set_properties(**{"text-align":"center"})
  #LEFT ALIGN COMMENTS COLUMN
  table=table.set_properties(subset=[" "],**{"text-align":"left"})
  #COLUMN HEADING - MAX WIDTH 200PX & CENTRE ALIGN
  #ROW HEADING - RIGHT ALIGN
  table=table.set_table_styles(
      [dict(selector=".col_heading",
            props=[("max-width","200px"),("text-align","center")]),
       dict(selector=".row_heading",
            props=[("text-align","right")])])

  display(table)

# FINAL PRODUCT

In [121]:
#BUTTON WIDGET
calculate_target_button=widgets.Button(description="Calculate Target")

#OUTPUT WIDGET
results=widgets.Output()

#SPACER WIDGET
spacer=widgets.HTML(value="<br>")

#TARGET MODEL & RESULTS
@results.capture(clear_output=True)
def target_model_results(_):
  #REGISTER FIRST CLICK
  fc_widget.value=True
  #USER INPUTS DO NOT MEET IMPOSED CONDITIONS
  if not user_inputs_check():
    #ERROR MESSAGE
    display(user_inputs_error)
  #USER INPUTS MEET IMPOSED CONDITIONS
  else:
    #TARGET MODEL
    global ip,target
    ip=user_inputs_convert(user_inputs_capture())
    target=target_model()
    #RESULTS
    display(results_header)
    display(spacer)
    sector_plot()
    display(spacer)
    target_table()
    display(spacer)
    company_target_plot()
    display(spacer)
    data_table()

#LINK BUTTON TO FUNCTION
calculate_target_button.on_click(target_model_results)

#DISPLAY
_=VBox([
    #USER INPUT FORM
    overall_header,
    basic_info_header,
    basic_info_input,
    by_emissions_header,
    core_boundary,
    by_emissions_input,
    by_activity_header,
    by_activity_input,
    ty_activity_header,
    ty_activity_input,
    ty_activity_footer,
    #TARGET MODEL & RESULTS
    calculate_target_button,
    results],
    layout={"width":"1110px"})
display(_)